In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# Load data using the correct delimiter
df = pd.read_csv('lilac_schedule.csv', delimiter=';')

# Verify and select the correct columns
columns_to_keep = ['Time (EST)', 'Location', 'Event', 'Speaker']
df = df[columns_to_keep]

# Replace NaNs with '--'
df.fillna('--', inplace=True)

# Convert DataFrame to HTML
html_table = df.to_html(index=False, border=0, escape=False, classes='schedule-table')

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_table, 'html.parser')

# Find rows containing days of the week
days_of_week = ['Monday, May 6', 'Tuesday, May 7', 'Wednesday, May 8', 'Thursday, May 9', 'Friday, May 10']
for day in days_of_week:
    for row in soup.find_all('tr'):
        if day in str(row):
            # Bold and center the day row and merge all its columns into one
            row_td = row.find_all('td')
            if row_td:
                merged_content = f'<td colspan="{len(row_td)}" style="text-align: center; font-weight: bold;">{day}</td>'
                row.clear()  # Clear existing td elements
                row.append(BeautifulSoup(merged_content, 'html.parser'))

# Re-convert to HTML
html_table = str(soup)

# Build the HTML content for your participants page with the table
html_content = f"""---
layout: default
title: Schedule
order: 2
---

# Schedule

- The meeting is intended to be centered on discussion and hands-on work.
- All times are in EST and the in-person location is Pratt unless otherwise marked.
- The Zoom link will be shared with the participants.

[Here is the schedule and slides for flash talks.](https://docs.google.com/presentation/d/1odfo0UHaLei6cYr-or-fq1fBINV75oakTbvppaN3BgI/edit?usp=sharing)

{html_table}
"""

# Write the HTML content to a file
with open('lilac_schedule.html', 'w') as file:
    file.write(html_content)
